In [13]:
# flask
from flask import Flask, render_template, url_for, request
from flask_ngrok import run_with_ngrok

# imports
import pandas as pd
import nltk
import matplotlib.pyplot as plt
import numpy as np
from nltk.corpus import stopwords
from nltk.stem import *
from nltk.stem.snowball import SnowballStemmer
from nltk.stem import WordNetLemmatizer
import joblib

In [14]:
# preprocessing

# tokenization Body
def Tokenization_texte(texte):
    x = []
    tokenizer = nltk.RegexpTokenizer(r'\w+')
    for row in texte:
        x.append(tokenizer.tokenize(row))
    return x

# stemmatization Body
def text_stematization(texte):
    stemmer = SnowballStemmer("english")
    stem_word = []
    stem_sentence = []
    for i in range(len(texte)):
        for word in texte[i]:
            stem_word.append(stemmer.stem(word))
        stem_sentence.append(stem_word)
        stem_word = [] 
    return stem_sentence


# lemmatization Body
def text_lemmatization(texte):
    lemmatizer = WordNetLemmatizer()
    lem_word = []
    lem_sentence = []
    for i in range(len(texte)):
        for word in texte[i]:
            lem_word.append(lemmatizer.lemmatize(word))
        lem_sentence.append(lem_word)
        lem_word = [] 
    return lem_sentence
    
# removing stopwords Body
def delete_stopwords(texte):
    cleaned_sentence = []
    cleaned_texte = []
    for sentence in texte:
        for word in sentence:
            if word not in stopwords.words('english'):
                cleaned_sentence.append(word)
        cleaned_texte.append(cleaned_sentence)
        cleaned_sentence = []
    return cleaned_texte

# removing frequent_words Body
def delete_frequent_words(texte,all_words_df):
    
    frequent_words = list(all_words_Body_df.text_words.value_counts().sort_values(ascending=False)[0:50].index)
    to_keep = ['string', 'http', 'java', 'android', 'list']
    for word in to_keep:
        frequent_words.remove(word)
    cleaned_sentence = []
    cleaned_texte = []
    for sentence in texte:
        for word in sentence:
            if word not in frequent_words:
                cleaned_sentence.append(word)
        cleaned_texte.append(cleaned_sentence)
        cleaned_sentence = []
    return cleaned_texte


def text_reconstruction(texte):
    return " ".join([word for word in texte])  

# text_cleaning Body
def text_cleaning(texte,all_words_df):
    cleaned_texte = Tokenization_texte(texte)
    cleaned_texte = text_stematization(cleaned_texte)
    cleaned_texte = text_lemmatization(cleaned_texte)
    cleaned_texte = delete_stopwords(cleaned_texte)
    cleaned_texte = delete_frequent_words(cleaned_texte,all_words_df)
    reconstructed_text = []
    for i in range(len(cleaned_texte)):
        reconstructed_sentence = text_reconstruction(cleaned_texte[i])
        reconstructed_text.append(reconstructed_sentence)
        reconstructed_sentence = []
    return reconstructed_text

In [15]:
def predict_categorie(predictions):
        cat=['java', 'c#', 'javascript', 'python', 'jquery', 'html', 'php',
       'android', '.net', 'c++', 'sql', 'css', 'mysql', 'string', 'sql-server',
       'c', 'git', 'arrays', 'asp.net', 'linux', 'regex', 'iphone', 'bash',
       'objective-c', 'ios', 'windows', 'ruby', 'eclipse', 'tsql', 'database']
        predicted_cat = []
        empty_list = []
        for i in range((len(cat)-1)):
            if predictions[i][0]==1:
                predicted_cat.append(cat[i])
        if predicted_cat == empty_list:
            predicted_cat.append('unknown')
        return predicted_cat

In [62]:
df = joblib.load('cleaned_data.sav')
all_words_Body_df = pd.read_csv('/home/abdifatah/Ingenieur_Machine_Learning/all_words_Body_df.csv',sep=',')
#model = joblib.load('final_model_v2.joblib')

In [63]:
from sklearn.preprocessing import MultiLabelBinarizer
mlb = MultiLabelBinarizer()
X = df.Body
Y = df.drop(columns = ['Id','index','Body','Tags'])

In [64]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.33, random_state=42)

In [79]:
from sklearn.svm import LinearSVC
# création de notre model
class custom_model:   
    # initialisation
    def __init__(self,cat = ['java', 'c#', 'javascript', 'python', 'jquery', 'html', 'php',
       'android', '.net', 'c++', 'sql', 'css', 'mysql', 'string', 'sql-server',
       'c', 'git', 'arrays', 'asp.net', 'linux', 'regex', 'iphone', 'bash',
       'objective-c', 'ios', 'windows', 'ruby', 'eclipse', 'tsql', 'database']):
        self.model = []
        self.categories = cat
        for i in range(len(cat)):
            self.model.append(LinearSVC(max_iter=5000))
    def fit(self,X,y):
        for i in range(len(self.model)):
            self.model[i].fit(X,y[self.categories[i]])
    def predict(self,X):
        predictions = []
        for i in range(len(self.model)):
            predictions.append(self.model[i].predict(X))
        return (predictions)

In [80]:
classifier = custom_model()

In [81]:
#pipeline
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
prediction_pipeline = Pipeline([('tf_idf',TfidfVectorizer(max_df = 0.95, min_df = 0.01 , stop_words = 'english')),('model',classifier)])

In [82]:
prediction_pipeline.fit(X_train,y_train)

Pipeline(steps=[('tf_idf',
                 TfidfVectorizer(max_df=0.95, min_df=0.01,
                                 stop_words='english')),
                ('model', <__main__.custom_model object at 0x7f3a0261ceb0>)])

In [83]:
def make_prediction(title, body):
    text = []
    text.append(title+''+body)
    cleaned_text = text_cleaning(text,all_words_Body_df)
    prediction = prediction_pipeline.predict(cleaned_text)
    prediction = predict_categorie(prediction)
    return prediction

In [84]:
make_prediction('What is the simplest way to write web apps in Haskell?', '<p>I would like to use Haskell more for my projects, and I think if I can get started using it for web apps, it would really help that cause. I have tried happs once or twice but had trouble getting off the ground. Are there simpler/more conventional (more like lamp) frameworks out there that I can use or should I just give happs another try?</p>\n')

['unknown']

In [85]:
app = Flask(__name__)
run_with_ngrok(app)
 
@app.route('/', methods = ['GET','POST'])
def main():
    return """<!DOCTYPE html>
                <html>
                    <head>
                         <title>Catégoriser questions</title>
                    </head>
                    <body>
                        <div align="center" class="bg-info">
                            <h1>Prediction de Tags</h1>
                        </div>
                        <div class="big" align="center">
                            <form action="predict" method="POST">
                                <h3>Entrez un titre</h3>
                                <textarea name="title" rows="1" cols="70"></textarea>
                                <br>
                                <h3>Entez un contenu</h3>
                                <textarea name="body" rows="20" cols="70"></textarea>
                                   <br><br><br>
                               <input type="submit" name="" value="Predict" class="btn btn-info">
                              </form>
                         </div>
                    </body>
                </html>
                """

@app.route('/predict', methods = ['POST'])
def predict():
    title = ""
    body = ""
    if request.method == 'POST':
        title = str(request.form['title'])
        body = str(request.form['body'])
    tags = make_prediction(title, body)
    return """
            <!DOCTYPE html>
            <html>
                <head>
                     <title>Catégoriser questions</title>
                </head>
                <body>
                    <div class="big" align="center">
                                <h4>Titre saisi</h4>
                                <textarea name="title" rows="1" cols="100"disabled>"""+ title +"""</textarea>
                                <br>
                                <h4>Contenu saisi</h4>
                                <textarea name="body" rows="7" cols="100" disabled>"""+ body +"""</textarea>
                                <br><br>
                     </div>
                     <div align="center">
                          <h2>Tags proposes :</h2>
                          <textarea rows="5" cols="50" disabled>"""+ str(tags) +"""</textarea>
                     </div>
                     <br><br><br>
                     <form action="/" method="POST" align="center">
                         <input type="submit" name="" value="Try again" class="btn btn-info">
                     </form>
                </body>
            </html>
        """
if __name__ == '__main__':
    app.run()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
Exception in thread Thread-324:
Traceback (most recent call last):
  File "/home/abdifatah/anaconda3/lib/python3.9/site-packages/urllib3/connection.py", line 174, in _new_conn
    conn = connection.create_connection(
  File "/home/abdifatah/anaconda3/lib/python3.9/site-packages/urllib3/util/connection.py", line 73, in create_connection
    for res in socket.getaddrinfo(host, port, family, socket.SOCK_STREAM):
  File "/home/abdifatah/anaconda3/lib/python3.9/socket.py", line 954, in getaddrinfo
    for res in _socket.getaddrinfo(host, port, family, type, proto, flags):
socket.gaierror: [Errno -3] Temporary failure in name resolution

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/home/abdifatah/anaconda3/lib/python3.9/site-packages/urllib3/connectionpool.py", line 699, in urlopen
    httplib_response = self._make_request(
  File "/home/abdifatah/anaconda3/lib/pyth